In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import scipy.linalg as linalg
from scipy.stats import multivariate_normal

In [ ]:
%matplotlib inline

In [ ]:
# converting from NX50X50 array into NX2500 array
flatimages = list()
labels = list()
face_path = 'faces2'

for file in os.listdir(face_path):
    if file.endswith('.jpg'):
        class_id = int(file.split('_')[0].lstrip('c'))
        img = mpimg.imread(os.path.join(face_path, file))
        flatimages.append(img.ravel())
        labels.append(class_id)
X = np.asarray(flatimages)
y = np.asarray(labels)

nrow, ncol = img.shape
N = X.shape[0]

In [ ]:
X.shape, y.shape

In [ ]:
# shuffle images
ind = np.random.permutation(N)
X = X[ind,:]
y = y[ind]
X.shape, y.shape

In [ ]:
# check training vector and labels
plt.imshow(X[19].reshape(nrow,ncol), cmap='gray')
print ('label is ', y[19])

In [ ]:
# PCA
# 1. find mean
mu=np.mean(X,axis=0);print(mu.shape)

# 2. sample center to zero mean
Z=X-mu

# 3. find covariance matrix
C=np.cov(Z.T);print(C.shape)

# 4. find eigenvalue and eigenvector
[sigma,V]=linalg.eigh(C)


In [ ]:
# columns of V are eigenvectors, two eigenvectors (v1, v2)
# flip eigenvector matrix (left/right) and change eigenvalue order from big to small
sigma=np.flipud(sigma)
V=np.fliplr(V)

In [ ]:
# get first 50 eigenvector for model fitting
np.sum(sigma[:2])/np.sum(sigma)r

In [ ]:
P2 = np.dot(Z, V[:, 0:2])rrr

# plot first 2 principle components
plt.figure(figsize=(10,8))
ax = plt.gca()
ax.spines['top'].set_color('none')
ax.spines['bottom'].set_color('none')
ax.spines['left'].set_color('none')
ax.spines['right'].set_color('none')
ax.xaxis.set_ticks_position('bottom')
ax.spines['bottom'].set_position(('data', 0))
ax.spines['bottom'].set_color('gray')
ax.yaxis.set_ticks_position('left')
ax.spines['left'].set_position(('data', 0))
ax.spines['left'].set_color('gray')
plt.scatter(P2[:,0][y==0], P2[:,1][y==0], c='red', s = 10, label = 'fei', alpha=0.7)
plt.scatter(P2[:,0][y==1], P2[:,1][y==1], c='green', s = 10, label = 'weiya', alpha=0.7)
ax.legend()
plt.show()

In [ ]:
# plot first 6 eigenfaces
plt.figure(figsize=(10,8))
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(V[:,i].reshape(nrow,ncol), cmap='gray')
plt.show()

In [ ]:
#Bayesian modelr

# collect first 50 principle component
P2 = np.dot(Z, V[:,0:2])
print (P2.shape)

# sample counts of positive and negative class
N0 = np.sum(y==0)
N1 = np.sum(y==1)
print (N0, N1, len(y))

# training samples for positive and negative class
X0 = P2[(y==0), :]
X1 = P2[(y==1), :]
print (len(X0), len(X1))

# mean of positive and negative samples
mu0 = np.mean(X0, axis=0)
mu1 = np.mean(X1, axis=0)

# covariance matrices for positive and negative samples
c0 = np.cov(X0.T)
c1 = np.cov(X1.T)

In [ ]:
pdf0 = multivariate_normal.pdf(P2, mean=mu0, cov=c0)
pdf1 = multivariate_normal.pdf(P2, mean=mu1, cov=c1)
probs_0 = N0*pdf0/(N0*pdf0+N1*pdf1)
probs_1 = N1*pdf1/(N0*pdf0+N1*pdf1)

pred = np.zeros_like(y)
pred.fill(np.nan)
pred[probs_0>probs_1] = 0
pred[probs_0<probs_1] = 1

In [ ]:
TP = np.sum((pred==0)&(y==0))
TN = np.sum((pred==1)&(y==1))
FP = np.sum((pred==1)&(y==0))
FN = np.sum((pred==0)&(y==1))
TP, TN, FP, FN

In [ ]:
# save eigenvectors and model parameters to file
np.savez('mean_eigenvectors_sigma.npz', mu = mu, V = V, sigma = sigma)  

np.savez('class0_stats.npz', N0 = np.array([N0]), mu0 = mu0, c0 = c0)
np.savez('class1_stats.npz', N1 = np.array([N1]), mu1 = mu1, c1 = c1)